# 模型部署 使用flask python 
网址： 
https://www.lanqiao.cn/courses/40981/learning/?id=2813919&compatibility=false

# 原题
```python 
import json
import numpy as np
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify


app = Flask(__name__)

def generate_text(start_sequence, max_length):
    #TODO


    pass

@app.route('/search', methods=['POST'])
def search():
    #TODO
    
    
    return jsonify(text=generated_text) # generated_text 是 generate_text 函数返回值

if __name__ == '__main__':
    app.run(debug=True)
```
---
# 手打题目
```python 
import json
import numpy as np
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify

app = Flask(__name__)

def generate_text(start_sequence, max_length):
    # 加载训练好的语言模型
    model = load_model("lm.h5")
    # 加载词汇表和索引转换字典
    with open("vocab.json", 'r', encoding='utf-8') as f:
        vocab_data = json.load(f)
        word_to_idx = vocab_data['word_to_idx']
        idx_to_word = vocab_data['idx_to_word']

    generated_text = start_sequence
    input_seq = [word_to_idx[word] for word in generated_text.split()]
    input_seq = np.array(input_seq).reshape(1, -1)

    for _ in range(max_length):
        predictions = model.predict(input_seq)
        pred_idx = np.argmax(predictions)
        word = idx_to_word[str(pred_idx)]
        generated_text += " " + word
        input_seq = np.append(input_seq, pred_idx)
        input_seq = input_seq[1:].reshape(1, -1)

    generated_indexes = [word_to_idx[word] for word in generated_text.split()]
    return generated_indexes

@app.route('/search', methods=['POST'])
def search():
    data = request.get_json()
    start_sequence = data['start_sequence']
    max_length = data['max_length']  # 从请求的 JSON 数据中获取 max_length 值
    generated_text = generate_text(start_sequence, max_length)
    return jsonify(text=generated_text)

if __name__ == '__main__':
    app.run(debug=True)


```
<mark>ERROR</mark>: 
* 从请求体中取出目标变量值 data = request.get_json() 使用 start_sequence = data['start_sequence'] 注意引号不可省略
* with open("vocab.json", 'r', encoding='utf-8') as f:
        vocab_data = json.load(f)
        word_to_idx = vocab_data['word_to_idx']
        idx_to_word = vocab_data['idx_to_word']
  读取一个json 文件，先打开后as file ，使用 json.load(file) ,同样的需要在使用时添加['str']
* 模型的输入是一个 np narray 类型 (input_seq 由查询得来) ，输出 是一个narray,在其中找到最大的对应的序号， predix = np.argmax(predictions), 对应的单词为 idx_to_word[str(predix)] 仅需将最初的输入与后续输出的单词 字符串拼接在一起即可 ，但是对于下次的输入 将改为 input_seq = np.append(input_seq, predix) 具体的修改，可能是去除历史第一个或第二个 input_seq = input_seq[2:].reshape(1,-1) 
---
# 参考答案
```python

```

# 模型部署 使用flask python 

介绍
在本任务中，你将使用训练好的语言模型作为底层模型之一来实现辅助用户搜索。一种实现方式是将语言模型封装为一个 Web API，提供对外的接口服务。通过定义 API 路由和请求处理逻辑，接收用户的输入请求并返回结果。

准备
开始答题前，请确认 /home/project 目录下包含以下文件：

lm.h5，是本任务提供的语言模型。你可以使用下面的代码查看模型。
```
import tensorflow
model = tensorflow.keras.models.load_model("lm.h5")
model.summary()
```
vocab.json，包含了词汇表和索引转换字典，可用于将文本中的词汇转换为语言模型能够理解的索引表示，形如：
```
{
  "word_to_idx": {
    "apple": 0,
    "banana": 1,
    "orange": 2,
    "grape": 3,
    "pear": 4
  },
  "idx_to_word": {
    "0": "apple",
    "1": "banana",
    "2": "orange",
    "3": "grape",
    "4": "pear"
  }
} 
```
task.py，是你后续答题过程中编写代码的地方。

目标
请按要求实现以下目标。

目标 1：实现 generate_text 函数

参数
start_sequence（字符串）：文本的起始词。
max_length（整数）：模型生成词的最大个数。
功能
加载提供的语言模型和词汇表。
使用语言模型生成文本，最多生成 max_length 个词。
返回值
generated_indexes（列表）：最终文本的索引表示，即每个词对应的索引（含起始词索引）。
目标 2：实现 search 函数

参数
无。
功能
处理 POST 请求，接收客户端发送的 JSON 数据，提取起始词（start_sequence）和最大词长（max_length）。
调用 generate_text 函数生成文本，以 JSON 格式响应给客户端。
返回值
JSON 数据，包含名为 text 的字段，其值为生成的文本的索引列表。
请求示例：
```
curl -X POST -H "Content-Type: application/json" -d '{"start_sequence": "生活", "max_length": 5}' http://localhost:5000/search
```
输出示例：
```
{
  "text": [
    12139,
    123,
    367,
    1200,
    5678,
    333
  ]
}
```
请在 task.py 文件中基于以下代码补充 #TODO 处的函数代码，确保能够实现目标。